In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import ElasticNet, ElasticNetCV, LassoCV, Lasso
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler, RobustScaler, scale
import seaborn as sns 

In [29]:
place_abbr = 'atlanta'

tracts_deserts_path = 'data/tract_desert_measures/atlanta_desert_tracts.csv'

save_data_dir = 'data/tract_health/'
save_data_name =  place_abbr+'_tract_health.csv'
save_transf_data_name =  place_abbr+'_tract_health_logt.csv'

save_regression_path = 'data/tract_health/results/'
save_lasso_name = place_abbr+'_tract_lasso_results'
save_enet_name = place_abbr+'_tract_enet_results'

# Aggregation/Cleaning

In [30]:
tracts_deserts = pd.read_csv(tracts_deserts_path, dtype={'GEOID':'str'})
tracts_deserts = tracts_deserts.rename(columns={'GEOID':'TractFIPS'})

In [31]:
tracts_health = pd.read_csv("data/PLACES__Census_Tract_Data__GIS_Friendly_Format___2021_release.csv", dtype={'TractFIPS':'str'})
tracts_health = pd.merge(tracts_health, tracts_deserts, on='TractFIPS')

income_vars = pd.read_csv('data/tract_incomevars.csv', dtype=str).astype({'PovertyRate':'float'})
tracts_health = pd.merge(tracts_health, income_vars, on='TractFIPS')

#tracts_health.to_csv(save_data_dir+save_data_name, index=False)

In [32]:
desert_measures = ['food_closest_travel_times', 'physical_closest_dist', 'transport_closest_dist', 'education_closest_travel_times', 'worship_closest_travel_times']

tracts_health_logt = tracts_health.copy()
tracts_health_logt[desert_measures] = tracts_health_logt[desert_measures].apply(lambda x: np.log(x+1))
#tracts_health_logt.to_csv(save_data_dir + save_transf_data_name, index=False)

# Analysis

In [33]:
name_mapping = {'access2': 'Health insurance access', 'arthritis': 'Arthritis prevalence', 'binge': 'Binge drinking prevalence',
               'bphigh': 'High blood pressure prevalence', 'bpmed': 'Medium blood pressure prevalence', 'cancer': 'Cancer prevalence',
               'casthma': 'Asthma prevalence', 'cervical': 'Cervical cancer screenings', 'chd': 'Coronary heart disease prevalence',
               'checkup': 'Routine checkups', 'cholscreen': 'Cholesterol screenings', 'colon_screen': 'Colon cancer screenings',
               'copd': 'COPD prevalence', 'corem': 'Core men\'s health', 'corew': 'Core women\'s health', 'csmoking': 'Smoking prevalence',
               'dental': 'Dental checkups', 'depression': 'Depression prevalence', 'diabetes': 'Diabetes prevalence', 'ghlth': 'General poor health prevalence',
               'highchol': 'High cholesterol prevalence', 'kidney': 'Chronic kidney disease', 'lpa': 'No physical activity', 'mammouse': 'Mammograms',
               'mhlth': 'Poor mental health prevalence', 'obesity': 'Obesity prevalence', 'phlth': 'Poor physical health', 'sleep': 'Poor sleep prevalence',
               'stroke': 'Stroke prevalence', 'teethlost': 'Teeth loss prevalence'}

results_all_nan = pd.DataFrame({'Health condition': [None for c in tracts_health.columns if c.endswith('CrudePrev')], 'Food': [None for c in tracts_health.columns if c.endswith('CrudePrev')],
                       'Physical health': [None for c in tracts_health.columns if c.endswith('CrudePrev')], 'Public transport': [None for c in tracts_health.columns if c.endswith('CrudePrev')],
                        'Education': [None for c in tracts_health.columns if c.endswith('CrudePrev')], 'Houses of worship': [None for c in tracts_health.columns if c.endswith('CrudePrev')],
                       'Poverty Rate': [None for c in tracts_health.columns if c.endswith('CrudePrev')], 'RSquared': [None for c in tracts_health.columns if c.endswith('CrudePrev')], 
                            'MSE': [None for c in tracts_health.columns if c.endswith('CrudePrev')]})
results_nan = pd.DataFrame({'Health condition': [None for c in tracts_health.columns if c.endswith('CrudePrev')], 'Food': [None for c in tracts_health.columns if c.endswith('CrudePrev')],
                       'Physical health': [None for c in tracts_health.columns if c.endswith('CrudePrev')], 'Poverty Rate': [None for c in tracts_health.columns if c.endswith('CrudePrev')], 
                            'RSquared': [None for c in tracts_health.columns if c.endswith('CrudePrev')], 
                            'MSE': [None for c in tracts_health.columns if c.endswith('CrudePrev')]})
add_vars = ['PovertyRate']

In [34]:
def update_all_vars():
        results.iat[i, 0] = name
        results.iat[i, 1] = regr.coef_[0]
        results.iat[i, 2] = regr.coef_[1]
        results.iat[i, 3] = regr.coef_[2]
        results.iat[i, 4] = regr.coef_[3]
        results.iat[i, 5] = regr.coef_[4]
        results.iat[i, 6] = regr.coef_[5]
        results.iat[i, 7] = regr.score(X_test, y_test)
        results.iat[i, 8] = mse_test
def update_vars():
        results.iat[i, 0] = name
        results.iat[i, 1] = regr.coef_[0]
        results.iat[i, 2] = regr.coef_[1]
        results.iat[i, 3] = regr.coef_[2]
        results.iat[i, 4] = regr.score(X_test, y_test)
        results.iat[i, 5] = mse_test

## Run Elastic Net Regression:
- For all deserts types; *all_deserts = True*
- Food and physical activity deserts only; *all_deserts = False*
- With log transformed desert measures; *log_transf = True*
- Without log transformed desert measures; *log_transf = False*

In [35]:
#set desert and log transformation params
all_deserts=True
log_transf = False

In [36]:
if all_deserts == True:
    desert_measures = ['food_closest_travel_times', 'physical_closest_dist', 'transport_closest_dist', 'education_closest_travel_times', 'worship_closest_travel_times']
    results = results_all_nan.copy()
    update_method = update_all_vars
else:
    desert_measures = ['food_closest_travel_times', 'physical_closest_dist']
    results = results_nan.copy()
    update_method = update_vars
if log_transf == True:
    df = tracts_health_logt
else:
    df = tracts_health
alphas = []
l1_ratios = []
i=0


for c in df.columns:
    if c.endswith('CrudePrev'):
        name = name_mapping[c[:-10].lower()]
        x = df[desert_measures+add_vars].to_numpy(copy=True)
        scaler = StandardScaler()
        xscale = scaler.fit_transform(x)
        y = df[c].to_numpy(copy=True)
        xscale = xscale[~np.isnan(y)]
        y = y[~np.isnan(y)]
        #create function to center data
        center_function = lambda x: x - x.mean()

        #apply function to original NumPy array
        data_centered = center_function(y)

        X_train, X_test, y_train, y_test = train_test_split(xscale, 
                                                    y, 
                                                    test_size=0.25, 
                                                    random_state=42)

        #l1 ratio is from suggested values in ElasticNetCV documentation
        enet_cv = ElasticNetCV(l1_ratio = [.1, .5, .7, .9, .95, .99, 1], 
                                     cv = 10, normalize=True).fit(X_train,y_train)

        alpha = enet_cv.alpha_
        l1 = enet_cv.l1_ratio_
        alphas.append(alpha)
        l1_ratios.append(l1)

        regr = ElasticNet(alpha=alpha, l1_ratio = l1, normalize=True)  # Could try others, or other parameters?
        regr.fit(X_train, y_train.reshape(-1, 1))

        predictions = regr.predict(X_test)
        y_train_pred = regr.predict(X_train)
        mse_test = mean_squared_error(y_test, predictions)

        if all_deserts==True:
            update_all_vars()
        else:
            update_vars()

        i += 1

results_round = results.round({'Food': 4, 'Physical health':4, 'Public transport':4, 'Education':4,
             'House of worship':4, 'RSquared':4})
results_round.sort_values(by='RSquared', ascending=False).reset_index(drop=True)

,Health condition,Food,Physical health,Public transport,Education,Houses of worship,Poverty Rate,RSquared,MSE
0,Dental checkups,-1.717107,-0.437347,-2.421626,1.406869,1.213129,-11.029982,0.825526,33.066618
1,Core men's health,-0.914028,-0.051717,-1.701063,1.45617,0.685964,-5.323313,0.802695,10.866408
2,Teeth loss prevalence,1.564003,0.391146,1.20176,-1.027328,-0.683275,8.570628,0.799777,17.585759
3,Core women's health,-0.492267,-0.0,-1.231694,0.757999,0.266511,-4.773899,0.796013,8.031973
4,Poor mental health prevalence,0.109684,-0.0,0.399489,-0.0,-0.359885,3.895335,0.78652,3.806377
5,Colon cancer screenings,-0.33009,-0.408476,-0.0,-0.90582,1.044911,-5.352444,0.77628,10.703035
6,Smoking prevalence,0.77894,0.179828,0.980843,-0.196003,-0.550438,4.924313,0.768227,8.590728
7,Poor sleep prevalence,0.931485,-0.107263,2.272739,-1.095804,-0.784802,4.631708,0.754103,10.632421
8,No physical activity,1.812326,0.703726,1.782739,-1.436346,-0.471692,7.376936,0.745734,22.115973
9,Obesity prevalence,1.635719,0.361353,2.532862,-2.116758,-0.353215,5.413572,0.74553,15.765302


## Run Elastic Net for Whole Dataset

In [10]:
if all_deserts == True:
    desert_measures = ['food_closest_travel_times', 'physical_closest_dist', 'transport_closest_dist', 'education_closest_travel_times', 'worship_closest_travel_times']
    results = results_all_nan.copy()
    update_method = update_all_vars
else:
    desert_measures = ['food_closest_travel_times', 'physical_closest_dist']
    results = results_nan.copy()
    update_method = update_vars
if log_transf == True:
    df = tracts_health_logt
else:
    df = tracts_health
alphas = []
l1_ratios = []
i=0


for c in df.columns:
    if c.endswith('CrudePrev'):
        name = name_mapping[c[:-10].lower()]
        x = df[desert_measures+add_vars].to_numpy(copy=True)
        scaler = StandardScaler()
        xscale = scaler.fit_transform(x)
        y = df[c].to_numpy(copy=True)
        xscale = xscale[~np.isnan(y)]
        y = y[~np.isnan(y)]
        #create function to center data
        center_function = lambda x: x - x.mean()

        #apply function to original NumPy array
        data_centered = center_function(y)

        X_train, X_test, y_train, y_test = train_test_split(xscale, 
                                                    y, 
                                                    test_size=0.25, 
                                                    random_state=42)

        #l1 ratio is from suggested values in ElasticNetCV documentation
        enet_cv = ElasticNetCV(l1_ratio = [.1, .5, .7, .9, .95, .99, 1], 
                                     cv = 10, normalize=True).fit(X_train,y_train)

        alpha = enet_cv.alpha_
        l1 = enet_cv.l1_ratio_
        alphas.append(alpha)
        l1_ratios.append(l1)

        regr = ElasticNet(alpha=alpha, l1_ratio = l1, normalize=True)  # Could try others, or other parameters?
        regr.fit(xscale, y.reshape(-1, 1))

        predictions = regr.predict(xscale)
        mse_test = mean_squared_error(y, predictions)

        if all_deserts==True:
            update_all_vars()
        else:
            update_vars()

        i += 1

results_round = results.round({'Food': 4, 'Physical health':4, 'Public transport':4, 'Education':4,
             'House of worship':4, 'RSquared':4})
results_round.sort_values(by='RSquared', ascending=False).reset_index(drop=True)

,Health condition,Food,Physical health,Public transport,Education,Houses of worship,Poverty Rate,RSquared,MSE
0,Dental checkups,-1.5734,-0.4846,-2.5733,1.4508,1.172120,-11.228835,0.8310,35.975211
1,Core women's health,-0.3737,-0.0587,-1.0010,0.8545,0.222341,-4.775942,0.8125,7.297878
2,Core men's health,-0.7684,-0.0795,-1.6883,1.3514,0.666473,-5.483587,0.8094,11.745655
3,Teeth loss prevalence,1.5656,0.5597,1.2505,-1.1605,-0.723866,8.447997,0.8033,16.176696
4,Colon cancer screenings,-0.2783,-0.3111,-0.0211,-0.9891,0.844452,-5.703419,0.7941,12.630291
5,Poor mental health prevalence,0.0643,0.0000,0.4024,-0.0000,-0.291049,3.894353,0.7909,3.510895
6,Smoking prevalence,0.6892,0.2306,1.0221,-0.1813,-0.523050,5.070937,0.7742,9.684669
7,Poor sleep prevalence,0.7862,-0.0919,2.2433,-1.1094,-0.699586,4.720244,0.7611,10.759051
8,Cervical cancer screenings,0.2113,0.0472,0.5838,-0.5602,0.287779,-2.788034,0.7543,5.180813
9,Obesity prevalence,1.4749,0.4592,2.5770,-2.1725,-0.428888,5.457041,0.7517,18.031118


### Save Output:

In [11]:
if all_deserts == True:
    save_enet_name += '_all'
if log_transf == True:
    save_enet_name += '_logt'
#results_round.to_csv(save_regression_path+save_enet_name+'.csv', index=False)

## Run LASSO regression:
- For all deserts types; *all_deserts = True*
- Food and physical activity deserts only; *all_deserts = False*
- With log transformed desert measures; *log_transf = True*
- Without log transformed desert measures; *log_transf = False*

In [12]:
#set desert and log transformation params
all_deserts=True
log_transf = False

In [13]:
if all_deserts == True:
    desert_measures = ['food_closest_travel_times', 'physical_closest_dist', 'transport_closest_dist', 'education_closest_travel_times', 'worship_closest_travel_times']
    results = results_all_nan.copy()
    update_method = update_all_vars
else:
    desert_measures = ['food_closest_travel_times', 'physical_closest_dist']
    results = results_nan.copy()
    update_method = update_vars
if log_transf == True:
    df = tracts_health_logt
else:
    df = tracts_health
alphas = []
l1_ratios = []
i=0


for c in df.columns:
    if c.endswith('CrudePrev'):
        name = name_mapping[c[:-10].lower()]
        
        x = df[desert_measures+add_vars].to_numpy(copy=True)
        scaler = StandardScaler()
        xscale = scaler.fit_transform(x)
        y = df[c].to_numpy(copy=True)
        xscale = xscale[~np.isnan(y)]
        y = y[~np.isnan(y)]
        #create function to center data
        center_function = lambda x: x - x.mean()

        #apply function to original NumPy array
        y = center_function(y)
        
        X_train, X_test, y_train, y_test = train_test_split(xscale, 
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state=42)
        
        #l1 ratio is from suggested values in ElasticNetCV documentation
        lasso_cv = LassoCV(cv = 5, normalize=True).fit(X_train,y_train)
        
        alpha = lasso_cv.alpha_
        alphas.append(alpha)
        
        regr = Lasso(alpha=alpha, normalize=True)  # Could try others, or other parameters?
        regr.fit(X_train, y_train.reshape(-1, 1))
        
        predictions = regr.predict(X_test)
        y_train_pred = regr.predict(X_train)
        mse_test = mean_squared_error(y_test, predictions)
        
        if all_deserts==True:
            update_all_vars()
        else:
            update_vars()

        i += 1

results_round = results.round({'Food': 4, 'Physical health':4, 'Public transport':4, 'Education':4,
             'House of worship':4, 'RSquared':4})
results_round.sort_values(by='RSquared', ascending=False).reset_index(drop=True)

,Health condition,Food,Physical health,Public transport,Education,Houses of worship,Poverty Rate,RSquared,MSE
0,Dental checkups,-1.6687,-0.1662,-2.7362,1.1203,1.269844,-11.292831,0.8313,30.223642
1,Core men's health,-0.8304,0.0000,-1.8879,1.2128,0.785663,-5.570152,0.8160,9.705229
2,Teeth loss prevalence,1.6489,0.2988,1.3319,-1.1023,-0.618246,8.596461,0.8082,15.913019
3,Poor mental health prevalence,0.1275,-0.0000,0.5534,0.0000,-0.491068,3.860119,0.7968,3.524493
4,Core women's health,-0.1654,-0.0000,-1.3022,0.5375,0.291609,-4.990628,0.7826,8.654552
5,Colon cancer screenings,-0.3431,-0.3246,-0.1684,-0.9959,1.164299,-5.417353,0.7701,10.184588
6,Smoking prevalence,0.8126,0.1222,1.2280,-0.1908,-0.743024,4.811760,0.7680,8.225432
7,Poor sleep prevalence,0.8880,-0.2568,2.7134,-0.9559,-1.015347,4.818751,0.7604,9.920658
8,No physical activity,1.9502,0.5986,2.0764,-1.4533,-0.553475,7.459476,0.7550,20.974224
9,Obesity prevalence,1.7224,0.1975,2.9169,-2.1420,-0.474477,5.435222,0.7523,15.095042


## LASSO on Whole Dataset

In [14]:
#set desert and log transformation params
all_deserts=False
log_transf = False

In [15]:
if all_deserts == True:
    desert_measures = ['food_closest_travel_times', 'physical_closest_dist', 'transport_closest_dist', 'education_closest_travel_times', 'worship_closest_travel_times']
    results = results_all_nan.copy()
    update_method = update_all_vars
else:
    desert_measures = ['food_closest_travel_times', 'physical_closest_dist']
    results = results_nan.copy()
    update_method = update_vars
if log_transf == True:
    df = tracts_health_logt
else:
    df = tracts_health
alphas = []
l1_ratios = []
i=0


for c in df.columns:
    if c.endswith('CrudePrev'):
        name = name_mapping[c[:-10].lower()]
        
        x = df[desert_measures+add_vars].to_numpy(copy=True)
        scaler = StandardScaler()
        xscale = scaler.fit_transform(x)
        y = df[c].to_numpy(copy=True)
        xscale = xscale[~np.isnan(y)]
        y = y[~np.isnan(y)]
        #create function to center data
        center_function = lambda x: x - x.mean()

        #apply function to original NumPy array
        y = center_function(y)
        
        X_train, X_test, y_train, y_test = train_test_split(xscale, 
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state=42)
        
        #l1 ratio is from suggested values in ElasticNetCV documentation
        lasso_cv = LassoCV(cv = 5, normalize=True).fit(X_train,y_train)
        
        alpha = lasso_cv.alpha_
        alphas.append(alpha)
        
        regr = Lasso(alpha=alpha, normalize=True)  # Could try others, or other parameters?
        regr.fit(xscale, y.reshape(-1, 1))
        
        predictions = regr.predict(xscale)
        mse_test = mean_squared_error(y, predictions)
        
        if all_deserts==True:
            update_all_vars()
        else:
            update_vars()

        i += 1

results_round = results.round({'Food': 4, 'Physical health':4, 'Public transport':4, 'Education':4,
             'House of worship':4, 'RSquared':4})
results_round.sort_values(by='RSquared', ascending=False).reset_index(drop=True)

,Health condition,Food,Physical health,Poverty Rate,RSquared,MSE
0,Dental checkups,-1.0411,-0.8789,-12.114793,0.8177,40.534343
1,Poor mental health prevalence,0.0680,0.0021,3.886646,0.7977,3.732672
2,Teeth loss prevalence,1.1770,0.5418,9.057524,0.7948,17.748168
3,Core women's health,-0.0236,-0.1484,-5.211543,0.7931,8.395848
4,Core men's health,-0.2912,-0.2104,-6.200382,0.7845,14.200111
5,Smoking prevalence,0.6921,0.6200,5.265914,0.7722,10.466167
6,Colon cancer screenings,-0.5206,-0.3399,-5.614401,0.7716,13.295834
7,No physical activity,1.2913,1.2074,8.186346,0.7357,23.546372
8,Health insurance access,0.7308,1.0192,6.844976,0.7202,19.693220
9,General poor health prevalence,1.3826,1.2250,7.995301,0.7194,25.369248


### Save Output:

In [16]:
if all_deserts == True:
    save_lasso_name += '_all'
if log_transf == True:
    save_lasso_name += '_logt'
#results_round.to_csv(save_regression_path+save_lasso_name+'.csv', index=False)